# CycleGAN for Adverse Weather Image Translation: Implementation and Analysis
## 0. Introduction and Overview
CycleGAN is a powerful unpaired image-to-image translation technique that allows us to transform images from one domain to another without requiring paired training data. This notebook implements a CycleGAN model for translating between normal weather conditions and adverse weather conditions (snow, fog, night, or rain) using the ACDC (Adverse Conditions Dataset with Correspondences) dataset.

The primary objective is to:

- Generate realistic adverse weather images from clear weather images
- Restore clear weather images from adverse weather conditions
- Provide comprehensive evaluation and high-resolution inference capabilities

**The implementation follows the original CycleGAN paper by Zhu et al. ("Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks") with some improvements to enhance stability and generation quality.**

## 1. Dataset Preparation and Loading
### 1.1 Dataset Download
This section handles downloading the ACDC dataset from Google Drive. The ACDC dataset contains image pairs of driving scenes captured under normal conditions and various adverse weather conditions, including fog, snow, night, and rain.



In [ ]:
import os

# Define the path where you want to download and extract the dataset
# Change this to your preferred location
download_dir = "./downloaded_dataset"
extract_dir = "./extracted_dataset"

# Create directories if they don't exist
os.makedirs(download_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)

print(f"Download directory: {os.path.abspath(download_dir)}")
print(f"Extract directory: {os.path.abspath(extract_dir)}")

Download directory: /content/downloaded_dataset
Extract directory: /content/extracted_dataset


In [ ]:
import gdown

# The Google Drive URL you provided
drive_url = "https://drive.google.com/drive/folders/1_gI7gD-5zA0Gl9IOnRV_jfgVz4zm-nej?usp=sharing"

# Extract the folder ID from the URL
folder_id = drive_url.split("/")[-1].split("?")[0]
print(f"Folder ID: {folder_id}")

# Download the folder recursively
output_path = os.path.join(download_dir, "dataset_archive.zip")

# Method 1: Try downloading as a folder
try:
    print("Attempting to download as a folder...")
    gdown.download_folder(drive_url, output=download_dir, quiet=False)
    print("Folder download completed!")
    folder_downloaded = True
except Exception as e:
    print(f"Folder download failed: {e}")
    folder_downloaded = False

# Method 2: If folder download fails, try downloading as a single file
if not folder_downloaded:
    try:
        print("Attempting to download as a single file...")
        # Format URL for single file download
        file_url = f"https://drive.google.com/uc?id={folder_id}&export=download"
        gdown.download(file_url, output_path, quiet=False)
        print(f"File downloaded to: {output_path}")
    except Exception as e:
        print(f"Single file download failed: {e}")
        print("You may need to manually download this dataset.")

Folder ID: 1_gI7gD-5zA0Gl9IOnRV_jfgVz4zm-nej
Attempting to download as a folder...


Retrieving folder contents


Processing file 1RQ9gMu7UosgGwWViZXpFxR5I-AWS9Q-2 gt_detection_trainval_ref.zip
Processing file 1w186yVWV8dVPSETR9EVO_IM_gc3ScMlr gt_detection_trainval.zip
Processing file 1yIkoPv4WXkMpbkLu_V6_Dbi2js6ggSue rgb_anon_trainvaltest.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1RQ9gMu7UosgGwWViZXpFxR5I-AWS9Q-2
To: /content/downloaded_dataset/gt_detection_trainval_ref.zip
100%|██████████| 1.13M/1.13M [00:00<00:00, 11.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w186yVWV8dVPSETR9EVO_IM_gc3ScMlr
To: /content/downloaded_dataset/gt_detection_trainval.zip
100%|██████████| 4.17M/4.17M [00:00<00:00, 27.5MB/s]


Folder download failed: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1yIkoPv4WXkMpbkLu_V6_Dbi2js6ggSue

but Gdown can't. Please check connections and permissions.
Attempting to download as a single file...
Single file download failed: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1

In [ ]:
import zipfile
import glob
import shutil

download_dir = '/content/drive/MyDrive/ACDC_datasets'
# Look for all zip files in the download directory
zip_files = glob.glob(os.path.join(download_dir, "*.zip"))

if zip_files:
    for zip_file in zip_files:
        print(f"Extracting: {zip_file}")
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
    print(f"All zip files have been extracted to: {extract_dir}")
elif folder_downloaded:
    print("Data was downloaded as individual files, no extraction needed.")
else:
    print("No zip files found to extract.")

# List the extracted contents
print("\nExtracted contents:")
if os.path.exists(extract_dir):
    for item in os.listdir(extract_dir):
        print(f" - {item}")

Extracting: /content/drive/MyDrive/ACDC_datasets/gt_detection_trainval_ref.zip
Extracting: /content/drive/MyDrive/ACDC_datasets/gt_detection_trainval.zip
Extracting: /content/drive/MyDrive/ACDC_datasets/rgb_anon_trainvaltest.zip
All zip files have been extracted to: ./extracted_dataset

Extracted contents:
 - gt_detection
 - README.md
 - License.pdf
 - rgb_anon


### 1.2 Custom Dataset Implementation
The ACDCWeatherDataset class implements a PyTorch dataset that loads image pairs from the ACDC dataset. Key features include:

- Support for different weather conditions (fog, snow, night, rain)
- Training and validation splits
- Automatic pairing of normal (reference) and adverse weather images
- Preprocessing and data augmentation pipelines

### 1.3 Data Transformations
The transformation pipelines include:

- For training: resize, random cropping, horizontal flipping, color jittering, normalization
- For validation: resize and normalization
- All images are normalized to [-1, 1] range, which is standard for GAN training

In [ ]:
import os
import numpy as np
import random
import time
import datetime
import sys
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image

import wandb
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import itertools

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Configuration Cell
class Config:
    # Experiment Configuration
    experiment_name = "CycleGAN-ACDC-Adverse-Weather"
    condition = "snow"  # Options: 'fog', 'snow', 'night', 'rain'

    # Dataset Configuration
    dataset_root = "/content/extracted_dataset/"  # Replace with your ACDC dataset path
    image_size = 256
    batch_size = 1  # CycleGAN typically uses batch size of 1
    num_workers = 4

    # Model Configuration
    input_channels = 3
    output_channels = 3
    ngf = 64  # Number of generator filters in the first conv layer
    ndf = 64  # Number of discriminator filters in the first conv layer
    n_residual_blocks = 9  # Number of residual blocks in the generator
    init_type = 'normal'  # Network initialization [normal | xavier | kaiming | orthogonal]
    init_gain = 0.02  # Scaling factor for normal, xavier and orthogonal initialization

    # Training Configuration
    epochs = 200
    lr = 0.0002
    beta1 = 0.5  # Adam optimizer beta1
    beta2 = 0.999  # Adam optimizer beta2
    lambda_A = 10.0  # Weight for cycle loss (A -> B -> A)
    lambda_B = 10.0  # Weight for cycle loss (B -> A -> B)
    lambda_identity = 0.5  # Weight for identity loss

    # Learning rate scheduling
    lr_policy = 'linear'  # Options: [linear | step | plateau | cosine]
    lr_decay_iters = 50  # Epoch at which lr starts decaying (if using 'step')
    n_epochs = 100  # Number of epochs with initial learning rate
    n_epochs_decay = 100  # Number of epochs to linearly decay learning rate to zero

    # Logging Configuration
    save_freq = 10  # Save model every n epochs
    log_freq = 100  # Log images every n iterations
    checkpoint_dir = './checkpoints'
    sample_dir = './samples'

    # Visualization Configuration
    sample_images_interval = 10  # Visualize images every n epochs

    # Wandb Configuration
    use_wandb = True
    wandb_project = "acdc-cyclegan"
    wandb_entity = "22097845d-the-hong-kong-polytechnic-university"  # Your wandb username or team name

    # Device Configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create directories if they don't exist
    @staticmethod
    def setup():
        os.makedirs(Config.checkpoint_dir, exist_ok=True)
        os.makedirs(Config.sample_dir, exist_ok=True)

        # Create specific directories for the current experiment
        experiment_checkpoint_dir = os.path.join(Config.checkpoint_dir, Config.experiment_name)
        experiment_sample_dir = os.path.join(Config.sample_dir, Config.experiment_name)

        os.makedirs(experiment_checkpoint_dir, exist_ok=True)
        os.makedirs(experiment_sample_dir, exist_ok=True)

        return experiment_checkpoint_dir, experiment_sample_dir

# Dataset Implementation
class ACDCWeatherDataset(Dataset):
    def __init__(self, root_dir, split='train', condition='fog', transform=None):
        """
        Custom dataset for loading adverse weather / normal image pairs from ACDC
        Args:
            root_dir: Path to the root directory (containing rgb_anon)
            split: 'train' or 'val'
            condition: 'rain', 'fog', 'night', or 'snow'
            transform: Optional transforms to apply to images
        """
        self.root_dir = root_dir
        self.split = split
        self.condition = condition
        self.transform = transform

        # Define paths for weather and reference images
        self.weather_dir = os.path.join(root_dir, 'rgb_anon', condition, split)
        self.ref_dir = os.path.join(root_dir, 'rgb_anon', condition, f'{split}_ref')

        # Build pairs directly from file structure
        self.pairs = []

        # Walk through the weather directory to find all images
        for sequence_dir in os.listdir(self.weather_dir):
            sequence_path = os.path.join(self.weather_dir, sequence_dir)
            if not os.path.isdir(sequence_path):
                continue

            for img_name in os.listdir(sequence_path):
                if img_name.endswith('_rgb_anon.png'):
                    # Create the corresponding reference image path
                    ref_img_name = img_name.replace('_rgb_anon.png', '_rgb_ref_anon.png')
                    ref_img_path = os.path.join(self.ref_dir, sequence_dir, ref_img_name)

                    # Check if reference image exists
                    if os.path.exists(ref_img_path):
                        self.pairs.append({
                            'weather_image': os.path.join(sequence_path, img_name),
                            'ref_image': ref_img_path
                        })

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]

        # Load images
        weather_image = Image.open(pair['weather_image']).convert('RGB')
        ref_image = Image.open(pair['ref_image']).convert('RGB')

        # Apply transforms if available
        if self.transform:
            weather_image = self.transform(weather_image)
            ref_image = self.transform(ref_image)

        # For CycleGAN:
        # A is normal/clear weather (reference)
        # B is adverse weather condition (fog, snow, etc.)
        return {
            'A': ref_image,       # Normal weather (reference)
            'B': weather_image,   # Adverse weather
            'A_path': pair['ref_image'],
            'B_path': pair['weather_image']
        }

# Define transformation pipelines
def get_transforms(opt):
    # For training, we use data augmentation
    if opt.split == 'train':
        transform = transforms.Compose([
            transforms.Resize((opt.image_size + 30, opt.image_size + 30)),  # Resize with margin for random cropping
            transforms.RandomCrop(opt.image_size),                          # Random cropping for data augmentation
            transforms.RandomHorizontalFlip(),                             # Random horizontal flipping
            transforms.ColorJitter(brightness=0.1, contrast=0.1),          # Random color jittering
            transforms.ToTensor(),                                         # Convert to tensor
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))         # Normalize to [-1, 1]
        ])
    # For validation, we only resize and normalize
    else:
        transform = transforms.Compose([
            transforms.Resize((opt.image_size, opt.image_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    return transform

# Helper class for managing image buffer (used to update discriminators using a history of generated images)
class ImageBuffer:
    def __init__(self, buffer_size=50):
        self.buffer_size = buffer_size
        self.buffer = []
        self.num_imgs = 0

    def query(self, images):
        if self.buffer_size == 0:
            return images

        return_images = []
        for image in images:
            image = torch.unsqueeze(image.data, 0)

            # If the buffer is not full, add the current image to the buffer
            if self.num_imgs < self.buffer_size:
                self.buffer.append(image)
                self.num_imgs += 1
                return_images.append(image)
            else:
                # With 50% chance, use image from buffer and add new image to buffer
                if random.random() > 0.5:
                    random_id = random.randint(0, self.buffer_size - 1)
                    tmp = self.buffer[random_id].clone()
                    self.buffer[random_id] = image
                    return_images.append(tmp)
                else:
                    return_images.append(image)

        return torch.cat(return_images, 0)



## 2. Model Architecture and Components
### 2.1 Generator Network
The Generator uses a *ResNet-based architecture* as the official cycleGAN with:

- Initial reflection padding and convolution
- Downsampling blocks for feature extraction
- Multiple residual blocks for transformation
- Upsampling blocks using *bilinear upsampling* instead of transposed convolutions (for reduction of checkerboard artifacts)
- Final convolution with tanh activation

### 2.2 Discriminator Network
The Discriminator uses a PatchGAN architecture:

- Multiple convolutional layers with increasing filter sizes
- Instance normalization and LeakyReLU activations
- No fully connected layers (maintaining spatial information)
- Produces a matrix of predictions rather than a single scalar

### 2.3 Auxiliary Components
Additional components for the CycleGAN framework:

- Image buffer for storing previously generated images (stabilizes discriminator training)
- Weight initialization functions for better convergence
- Learning rate scheduling for gradual learning rate decay
- Custom loss functions for adversarial, cycle consistency, and identity preservation



In [ ]:

# Model Architecture

# ResNet block used in generator
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        # Set affine=True to enable learnable parameters for InstanceNorm2d
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features, affine=True),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features, affine=True)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator network (ResNet-based) with upsampling instead of deconvolutions
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, 7),
            nn.InstanceNorm2d(ngf, affine=True),
            nn.ReLU(inplace=True)
        ]

        # Downsampling (encoder)
        in_features = ngf
        out_features = in_features * 2
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features, affine=True),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features * 2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling (decoder) - MODIFIED to use upsampling instead of deconvolution
        out_features = in_features // 2
        for _ in range(2):
            model += [
                # Replace ConvTranspose2d with Upsample + Conv2d
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features, affine=True),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(ngf, output_nc, 7),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# Discriminator network (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self, input_nc, ndf=64):
        super(Discriminator, self).__init__()

        # A series of convolutions with instance normalization
        model = [
            nn.Conv2d(input_nc, ndf, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(ndf, ndf * 2, 4, stride=2, padding=1),
            nn.InstanceNorm2d(ndf * 2, affine=True),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1),
            nn.InstanceNorm2d(ndf * 4, affine=True),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        model += [
            nn.Conv2d(ndf * 4, ndf * 8, 4, padding=1),
            nn.InstanceNorm2d(ndf * 8, affine=True),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        # FCN classification layer
        model += [nn.Conv2d(ndf * 8, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        x = self.model(x)
        # Average pooling and flatten
        return x


## 3. CycleGAN Model Implementation
### 3.1 Model Structure
The CycleGAN model consists of:

1. Two generators: G_A2B (normal → adverse) and G_B2A (adverse → normal)
2. Two discriminators: D_A (for normal images) and D_B (for adverse images)
3. Multiple loss components that work together to ensure realistic and consistent translations

### 3.2 Loss Functions
The training objective includes several loss components:

- Adversarial loss: Makes generated images look realistic
- Cycle consistency loss: Ensures G_B2A(G_A2B(A)) ≈ A and G_A2B(G_B2A(B)) ≈ B
- Identity loss: Encourages generators to preserve content when input is already from the target domain
- The weighted sum of these losses guides the model toward the desired behavior

### 3.3 Optimization Process
The optimization process alternates between:

- Updating generators to minimize combined losses
- Updating discriminators to better distinguish real and fake images
- Learning rate scheduling for stable convergence
- Tracking and saving the best model based on validation performance



In [ ]:

# Weight initialization function
def init_weights(net, init_type='normal', init_gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                nn.init.normal_(m.weight.data, 0.0, init_gain)
            elif init_type == 'xavier':
                nn.init.xavier_normal_(m.weight.data, gain=init_gain)
            elif init_type == 'kaiming':
                nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                nn.init.orthogonal_(m.weight.data, gain=init_gain)
            else:
                raise NotImplementedError(f'Initialization method {init_type} is not implemented')

            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.constant_(m.bias.data, 0.0)

        # For BatchNorm and InstanceNorm - checking if weight and bias exist first
        elif classname.find('BatchNorm2d') != -1 or classname.find('InstanceNorm2d') != -1:
            if hasattr(m, 'weight') and m.weight is not None:
                nn.init.normal_(m.weight.data, 1.0, init_gain)
            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.constant_(m.bias.data, 0.0)

    net.apply(init_func)
    return net

# Learning rate scheduler
def get_scheduler(optimizer, config):
    if config.lr_policy == 'linear':
        def lambda_rule(epoch):
            lr_l = 1.0 - max(0, epoch - config.n_epochs) / float(config.n_epochs_decay + 1)
            return lr_l
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
    elif config.lr_policy == 'step':
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer, step_size=config.lr_decay_iters, gamma=0.1
        )
    elif config.lr_policy == 'plateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.2, threshold=0.01, patience=5
        )
    elif config.lr_policy == 'cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=config.n_epochs, eta_min=0
        )
    else:
        return NotImplementedError(f'Learning rate policy {config.lr_policy} is not implemented')
    return scheduler

# Loss functions
class GANLoss(nn.Module):
    def __init__(self, target_real_label=1.0, target_fake_label=0.0):
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        self.loss = nn.MSELoss()

    def get_target_tensor(self, prediction, target_is_real):
        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        return target_tensor.expand_as(prediction)

    def __call__(self, prediction, target_is_real):
        target_tensor = self.get_target_tensor(prediction, target_is_real)
        return self.loss(prediction, target_tensor)

# CycleGAN model class
class CycleGANModel():
    def __init__(self, config):
        self.config = config
        self.device = config.device

        # Define networks
        self.netG_A2B = Generator(config.input_channels, config.output_channels, config.ngf, config.n_residual_blocks)
        self.netG_B2A = Generator(config.input_channels, config.output_channels, config.ngf, config.n_residual_blocks)
        self.netD_A = Discriminator(config.input_channels, config.ndf)
        self.netD_B = Discriminator(config.input_channels, config.ndf)

        # Initialize networks
        self.netG_A2B = init_weights(self.netG_A2B, config.init_type, config.init_gain)
        self.netG_B2A = init_weights(self.netG_B2A, config.init_type, config.init_gain)
        self.netD_A = init_weights(self.netD_A, config.init_type, config.init_gain)
        self.netD_B = init_weights(self.netD_B, config.init_type, config.init_gain)

        # Move networks to configured device
        self.netG_A2B.to(self.device)
        self.netG_B2A.to(self.device)
        self.netD_A.to(self.device)
        self.netD_B.to(self.device)

        # Define loss functions
        self.criterionGAN = GANLoss().to(self.device)
        self.criterionCycle = nn.L1Loss()
        self.criterionIdentity = nn.L1Loss()

        # Image buffers for historical generated images
        self.fake_A_buffer = ImageBuffer()
        self.fake_B_buffer = ImageBuffer()

        # Optimizers
        self.optimizer_G = torch.optim.Adam(
            itertools.chain(self.netG_A2B.parameters(), self.netG_B2A.parameters()),
            lr=config.lr, betas=(config.beta1, config.beta2)
        )
        self.optimizer_D = torch.optim.Adam(
            itertools.chain(self.netD_A.parameters(), self.netD_B.parameters()),
            lr=config.lr, betas=(config.beta1, config.beta2)
        )

        # Learning rate schedulers
        self.scheduler_G = get_scheduler(self.optimizer_G, config)
        self.scheduler_D = get_scheduler(self.optimizer_D, config)

        # Training metrics
        self.loss_G_A2B = 0.0
        self.loss_G_B2A = 0.0
        self.loss_D_A = 0.0
        self.loss_D_B = 0.0
        self.loss_cycle_A = 0.0
        self.loss_cycle_B = 0.0
        self.loss_identity_A = 0.0
        self.loss_identity_B = 0.0

        # Best model tracking
        self.best_val_loss = float('inf')

    def set_input(self, input_data):
        self.real_A = input_data['A'].to(self.device)
        self.real_B = input_data['B'].to(self.device)
        self.image_paths = {'A': input_data['A_path'], 'B': input_data['B_path']}

    def forward(self):
        # Forward pass through generators
        self.fake_B = self.netG_A2B(self.real_A)  # G_A(A)
        self.rec_A = self.netG_B2A(self.fake_B)   # G_B(G_A(A))
        self.fake_A = self.netG_B2A(self.real_B)  # G_B(B)
        self.rec_B = self.netG_A2B(self.fake_A)   # G_A(G_B(B))

        # Identity mapping
        if self.config.lambda_identity > 0:
            self.idt_A = self.netG_B2A(self.real_A)  # G_B(A)
            self.idt_B = self.netG_A2B(self.real_B)  # G_A(B)

    def backward_D_basic(self, netD, real, fake):
        # Real
        pred_real = netD(real)
        loss_D_real = self.criterionGAN(pred_real, True)

        # Fake
        pred_fake = netD(fake.detach())
        loss_D_fake = self.criterionGAN(pred_fake, False)

        # Combined loss
        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()

        return loss_D

    def backward_D_A(self):
        fake_A = self.fake_A_buffer.query(self.fake_A)
        self.loss_D_A = self.backward_D_basic(self.netD_A, self.real_A, fake_A)

    def backward_D_B(self):
        fake_B = self.fake_B_buffer.query(self.fake_B)
        self.loss_D_B = self.backward_D_basic(self.netD_B, self.real_B, fake_B)

    def backward_G(self):
        lambda_A = self.config.lambda_A
        lambda_B = self.config.lambda_B
        lambda_idt = self.config.lambda_identity

        # Identity loss
        if lambda_idt > 0:
            self.loss_identity_A = self.criterionIdentity(self.idt_A, self.real_A) * lambda_A * lambda_idt
            self.loss_identity_B = self.criterionIdentity(self.idt_B, self.real_B) * lambda_B * lambda_idt
        else:
            self.loss_identity_A = 0
            self.loss_identity_B = 0

        # GAN loss D_A(G_B(B))
        self.loss_G_B2A = self.criterionGAN(self.netD_A(self.fake_A), True)

        # GAN loss D_B(G_A(A))
        self.loss_G_A2B = self.criterionGAN(self.netD_B(self.fake_B), True)

        # Forward cycle loss || G_B(G_A(A)) - A ||
        self.loss_cycle_A = self.criterionCycle(self.rec_A, self.real_A) * lambda_A

        # Backward cycle loss || G_A(G_B(B)) - B ||
        self.loss_cycle_B = self.criterionCycle(self.rec_B, self.real_B) * lambda_B

        # Combined loss and calculate gradients
        self.loss_G = self.loss_G_A2B + self.loss_G_B2A + self.loss_cycle_A + self.loss_cycle_B + self.loss_identity_A + self.loss_identity_B
        self.loss_G.backward()

    def optimize_parameters(self):
        # Forward
        self.forward()

        # G_A and G_B
        self.set_requires_grad([self.netD_A, self.netD_B], False)  # Ds require no gradients when optimizing Gs
        self.optimizer_G.zero_grad()  # Set G's gradients to zero
        self.backward_G()             # Calculate gradients for G
        self.optimizer_G.step()       # Update G's weights

        # D_A and D_B
        self.set_requires_grad([self.netD_A, self.netD_B], True)
        self.optimizer_D.zero_grad()  # Set D's gradients to zero
        self.backward_D_A()           # Calculate gradients for D_A
        self.backward_D_B()           # Calculate gradients for D_B
        self.optimizer_D.step()       # Update D's weights

    def set_requires_grad(self, nets, requires_grad=False):
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

    def update_learning_rate(self):
        # Update learning rates
        self.scheduler_G.step()
        self.scheduler_D.step()

        # Log current learning rates
        lr_G = self.optimizer_G.param_groups[0]['lr']
        lr_D = self.optimizer_D.param_groups[0]['lr']

        return lr_G, lr_D

    def get_current_losses(self):
        return {
            'G_A2B': self.loss_G_A2B.item() if isinstance(self.loss_G_A2B, torch.Tensor) else self.loss_G_A2B,
            'G_B2A': self.loss_G_B2A.item() if isinstance(self.loss_G_B2A, torch.Tensor) else self.loss_G_B2A,
            'D_A': self.loss_D_A.item() if isinstance(self.loss_D_A, torch.Tensor) else self.loss_D_A,
            'D_B': self.loss_D_B.item() if isinstance(self.loss_D_B, torch.Tensor) else self.loss_D_B,
            'cycle_A': self.loss_cycle_A.item() if isinstance(self.loss_cycle_A, torch.Tensor) else self.loss_cycle_A,
            'cycle_B': self.loss_cycle_B.item() if isinstance(self.loss_cycle_B, torch.Tensor) else self.loss_cycle_B,
            'identity_A': self.loss_identity_A.item() if isinstance(self.loss_identity_A, torch.Tensor) else self.loss_identity_A,
            'identity_B': self.loss_identity_B.item() if isinstance(self.loss_identity_B, torch.Tensor) else self.loss_identity_B
        }

    def get_current_visuals(self):
        return {
            'real_A': self.real_A,
            'fake_B': self.fake_B,
            'rec_A': self.rec_A,
            'real_B': self.real_B,
            'fake_A': self.fake_A,
            'rec_B': self.rec_B
        }

    def save_networks(self, epoch, is_best=False, checkpoint_dir=None):
        if checkpoint_dir is None:
            checkpoint_dir = os.path.join(self.config.checkpoint_dir, self.config.experiment_name)

        # Create checkpoint directory if it doesn't exist
        os.makedirs(checkpoint_dir, exist_ok=True)

        # Prepare models and optimizers state dict
        save_dict = {
            'epoch': epoch,
            'netG_A2B': self.netG_A2B.state_dict(),
            'netG_B2A': self.netG_B2A.state_dict(),
            'netD_A': self.netD_A.state_dict(),
            'netD_B': self.netD_B.state_dict(),
            'optimizer_G': self.optimizer_G.state_dict(),
            'optimizer_D': self.optimizer_D.state_dict(),
            'scheduler_G': self.scheduler_G.state_dict(),
            'scheduler_D': self.scheduler_D.state_dict(),
            'config': self.config.__dict__
        }

        # Save checkpoint
        if is_best:
            save_path = os.path.join(checkpoint_dir, 'best_model.pth')
        else:
            save_path = os.path.join(checkpoint_dir, f'epoch_{epoch}.pth')

        torch.save(save_dict, save_path)
        print(f"Checkpoint saved to {save_path}")

    def load_networks(self, checkpoint_path):
        if not os.path.exists(checkpoint_path):
            print(f"Checkpoint {checkpoint_path} does not exist.")
            return False

        # Load checkpoint
        checkpoint = torch.load(checkpoint_path, map_location=self.device)

        # Load models
        self.netG_A2B.load_state_dict(checkpoint['netG_A2B'])
        self.netG_B2A.load_state_dict(checkpoint['netG_B2A'])
        self.netD_A.load_state_dict(checkpoint['netD_A'])
        self.netD_B.load_state_dict(checkpoint['netD_B'])

        # Load optimizers
        self.optimizer_G.load_state_dict(checkpoint['optimizer_G'])
        self.optimizer_D.load_state_dict(checkpoint['optimizer_D'])

        # Load schedulers
        self.scheduler_G.load_state_dict(checkpoint['scheduler_G'])
        self.scheduler_D.load_state_dict(checkpoint['scheduler_D'])

        # Load epoch
        epoch = checkpoint['epoch']

        print(f"Checkpoint loaded from {checkpoint_path} at epoch {epoch}")
        return epoch


## 4. Training and Evaluation
Training Loop
The training loop includes:

1. Epoch-based training with logging and visualization
2. Alternating generator and discriminator updates
3. Learning rate scheduling based on training progress
4. Saving checkpoints and best models
5. Integration with Weights & Biases for experiment tracking

### 4.1 Evaluation Function
The evaluation function:

- Assesses model performance on the validation set
- Calculates generator, discriminator, and cycle consistency losses
- Provides metrics for model selection
- Visualizes generated images for qualitative assessment

### 4.2 Visualization Functions
Visualization functions include:

- Creating comparison figures with original, generated, and reconstructed images
- Generating sample visualizations at regular intervals
- Saving high-quality images for inspection
- Logging images to Weights & Biases for online tracking



In [ ]:

# Training and Evaluation Functions

def train_model(model, train_loader, val_loader, config, checkpoint_dir, sample_dir):
    # Initialize Weights & Biases
    if config.use_wandb:
        wandb.init(
            project=config.wandb_project,
            entity=config.wandb_entity,
            name=config.experiment_name,
            config=vars(config)
        )

    # Training loop
    start_epoch = 1
    total_iters = 0

    for epoch in range(start_epoch, config.epochs + 1):
        epoch_start_time = time.time()
        epoch_iter = 0

        # Training
        model.netG_A2B.train()
        model.netG_B2A.train()
        model.netD_A.train()
        model.netD_B.train()

        # Initialize lists to collect losses
        epoch_losses_G = []
        epoch_losses_D = []

        for batch_data in tqdm(train_loader, desc=f"Epoch {epoch}/{config.epochs}"):
            total_iters += 1
            epoch_iter += 1

            # Set model input
            model.set_input(batch_data)

            # Optimize parameters
            model.optimize_parameters()

            # Get current losses
            losses = model.get_current_losses()

            # Record epoch losses
            epoch_losses_G.append(losses['G_A2B'] + losses['G_B2A'])
            epoch_losses_D.append(losses['D_A'] + losses['D_B'])

            # Log losses and images to Wandb
            if config.use_wandb and total_iters % config.log_freq == 0:
                # Log losses
                wandb.log({
                    'iter': total_iters,
                    'loss_G_A2B': losses['G_A2B'],
                    'loss_G_B2A': losses['G_B2A'],
                    'loss_D_A': losses['D_A'],
                    'loss_D_B': losses['D_B'],
                    'loss_cycle_A': losses['cycle_A'],
                    'loss_cycle_B': losses['cycle_B'],
                    'loss_identity_A': losses['identity_A'],
                    'loss_identity_B': losses['identity_B']
                })

        # Calculate average epoch losses
        avg_loss_G = sum(epoch_losses_G) / len(epoch_losses_G) if epoch_losses_G else 0
        avg_loss_D = sum(epoch_losses_D) / len(epoch_losses_D) if epoch_losses_D else 0

        # Update learning rates
        lr_G, lr_D = model.update_learning_rate()

        # Print epoch summary
        time_elapsed = time.time() - epoch_start_time
        print(f"Epoch {epoch}/{config.epochs} completed in {time_elapsed:.2f}s | "
              f"Avg G Loss: {avg_loss_G:.4f}, Avg D Loss: {avg_loss_D:.4f} | "
              f"LR_G: {lr_G:.6f}, LR_D: {lr_D:.6f}")

        # Log epoch metrics to wandb
        if config.use_wandb:
            wandb.log({
                'epoch': epoch,
                'avg_loss_G': avg_loss_G,
                'avg_loss_D': avg_loss_D,
                'lr_G': lr_G,
                'lr_D': lr_D
            })

        # Validation
        if epoch % config.save_freq == 0 or epoch == config.epochs:
            # Evaluate on validation set
            val_losses = evaluate_model(model, val_loader, config)

            # Log validation metrics
            if config.use_wandb:
                wandb.log({
                    'epoch': epoch,
                    'val_loss_G': val_losses['G'],
                    'val_loss_D': val_losses['D'],
                    'val_loss_cycle': val_losses['cycle']
                })

            # Save model if it's the best so far
            if val_losses['G'] + val_losses['cycle'] < model.best_val_loss:
                model.best_val_loss = val_losses['G'] + val_losses['cycle']
                model.save_networks(epoch, is_best=True, checkpoint_dir=checkpoint_dir)
                print(f"New best model saved at epoch {epoch}")

            # Save checkpoint
            model.save_networks(epoch, is_best=False, checkpoint_dir=checkpoint_dir)

        # Visualize generated images
        if epoch % config.sample_images_interval == 0 or epoch == config.epochs:
            # Set model to eval mode for visualization
            model.netG_A2B.eval()
            model.netG_B2A.eval()

            # New: Create multiple sample visualizations (5 samples)
            if config.use_wandb:
                wandb_samples = {'epoch': epoch}
                # Get 5 different samples from validation set
                val_iter = iter(val_loader)

                # Log 5 examples to wandb
                for sample_idx in range(5):
                    try:
                        sample_batch = next(val_iter)
                    except StopIteration:
                        # If we run out of validation samples, reset the iterator
                        val_iter = iter(val_loader)
                        sample_batch = next(val_iter)

                    with torch.no_grad():
                        # Set model input
                        model.set_input(sample_batch)
                        model.forward()

                        # Get current visuals
                        visuals = model.get_current_visuals()

                        # Create comparison figures for this sample
                        fig_A2B = create_comparison_figure(visuals['real_A'], visuals['fake_B'], visuals['rec_A'],
                                                          title=f"Normal to {config.condition.capitalize()} (Epoch {epoch}, Sample {sample_idx+1})")
                        fig_B2A = create_comparison_figure(visuals['real_B'], visuals['fake_A'], visuals['rec_B'],
                                                          title=f"{config.condition.capitalize()} to Normal (Epoch {epoch}, Sample {sample_idx+1})")

                        # Add to wandb samples
                        wandb_samples[f"Normal to {config.condition}_{sample_idx+1}"] = wandb.Image(fig_A2B)
                        wandb_samples[f"{config.condition} to Normal_{sample_idx+1}"] = wandb.Image(fig_B2A)

                        # Close figures to save memory
                        plt.close(fig_A2B)
                        plt.close(fig_B2A)

                # Log all samples to wandb at once
                wandb.log(wandb_samples)

                # Save one example to disk
                val_iter = iter(val_loader)
                sample_batch = next(val_iter)
                with torch.no_grad():
                    model.set_input(sample_batch)
                    model.forward()
                    visuals = model.get_current_visuals()
                    save_visuals(visuals, epoch, sample_dir, config)
            else:
                # Just save one example to disk if not using wandb
                val_iter = iter(val_loader)
                sample_batch = next(val_iter)
                with torch.no_grad():
                    model.set_input(sample_batch)
                    model.forward()
                    visuals = model.get_current_visuals()
                    save_visuals(visuals, epoch, sample_dir, config)

    # Finish wandb logging
    if config.use_wandb:
        wandb.finish()

def evaluate_model(model, val_loader, config):
    # Set model to evaluation mode
    model.netG_A2B.eval()
    model.netG_B2A.eval()
    model.netD_A.eval()
    model.netD_B.eval()

    # Initialize lists to collect validation losses
    val_losses_G = []
    val_losses_D = []
    val_losses_cycle = []

    with torch.no_grad():
        for batch_data in tqdm(val_loader, desc="Validation"):
            # Set model input
            model.set_input(batch_data)

            # Forward pass only
            model.forward()

            # Calculate losses
            # GAN loss
            fake_B = model.fake_B
            fake_A = model.fake_A
            pred_fake_B = model.netD_B(fake_B)
            pred_fake_A = model.netD_A(fake_A)
            loss_G_A2B = model.criterionGAN(pred_fake_B, True)
            loss_G_B2A = model.criterionGAN(pred_fake_A, True)

            # Cycle loss
            rec_A = model.rec_A
            rec_B = model.rec_B
            loss_cycle_A = model.criterionCycle(rec_A, model.real_A) * config.lambda_A
            loss_cycle_B = model.criterionCycle(rec_B, model.real_B) * config.lambda_B

            # Identity loss
            if config.lambda_identity > 0:
                loss_identity_A = model.criterionIdentity(model.idt_A, model.real_A) * config.lambda_A * config.lambda_identity
                loss_identity_B = model.criterionIdentity(model.idt_B, model.real_B) * config.lambda_B * config.lambda_identity
            else:
                loss_identity_A = 0
                loss_identity_B = 0

            # Discriminator loss
            pred_real_A = model.netD_A(model.real_A)
            pred_real_B = model.netD_B(model.real_B)
            loss_D_A_real = model.criterionGAN(pred_real_A, True)
            loss_D_B_real = model.criterionGAN(pred_real_B, True)

            pred_fake_A = model.netD_A(fake_A.detach())
            pred_fake_B = model.netD_B(fake_B.detach())
            loss_D_A_fake = model.criterionGAN(pred_fake_A, False)
            loss_D_B_fake = model.criterionGAN(pred_fake_B, False)

            loss_D_A = (loss_D_A_real + loss_D_A_fake) * 0.5
            loss_D_B = (loss_D_B_real + loss_D_B_fake) * 0.5

            # Total losses
            loss_G = loss_G_A2B + loss_G_B2A + loss_cycle_A + loss_cycle_B + loss_identity_A + loss_identity_B
            loss_cycle = loss_cycle_A + loss_cycle_B
            loss_D = loss_D_A + loss_D_B

            # Collect losses
            val_losses_G.append(loss_G.item())
            val_losses_D.append(loss_D.item())
            val_losses_cycle.append(loss_cycle.item())

    # Calculate average validation losses
    avg_val_loss_G = sum(val_losses_G) / len(val_losses_G) if val_losses_G else 0
    avg_val_loss_D = sum(val_losses_D) / len(val_losses_D) if val_losses_D else 0
    avg_val_loss_cycle = sum(val_losses_cycle) / len(val_losses_cycle) if val_losses_cycle else 0

    # Print validation results
    print(f"Validation - Avg G Loss: {avg_val_loss_G:.4f}, Avg D Loss: {avg_val_loss_D:.4f}, "
          f"Avg Cycle Loss: {avg_val_loss_cycle:.4f}")

    # Return average validation losses
    return {
        'G': avg_val_loss_G,
        'D': avg_val_loss_D,
        'cycle': avg_val_loss_cycle
    }

# Visualization functions
def tensor_to_numpy(tensor):
    # Convert tensor to numpy array
    image = tensor.detach().cpu().numpy()
    image = (image + 1) / 2.0  # Denormalize from [-1, 1] to [0, 1]
    image = np.transpose(image, (1, 2, 0))  # CHW to HWC
    image = np.clip(image, 0, 1)
    return image

def create_comparison_figure(real, fake, rec, title="Comparison"):
    # Create a figure with subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Convert tensors to numpy arrays and denormalize
    real_np = tensor_to_numpy(real[0])
    fake_np = tensor_to_numpy(fake[0])
    rec_np = tensor_to_numpy(rec[0])

    # Plot images
    axes[0].imshow(real_np)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(fake_np)
    axes[1].set_title("Generated")
    axes[1].axis('off')

    axes[2].imshow(rec_np)
    axes[2].set_title("Reconstructed")
    axes[2].axis('off')

    plt.suptitle(title)
    plt.tight_layout()

    return fig

def save_visuals(visuals, epoch, sample_dir, config):
    # Create directory for samples if it doesn't exist
    os.makedirs(sample_dir, exist_ok=True)

    # Get a batch of images
    real_A = visuals['real_A'].detach().cpu()
    fake_B = visuals['fake_B'].detach().cpu()
    rec_A = visuals['rec_A'].detach().cpu()
    real_B = visuals['real_B'].detach().cpu()
    fake_A = visuals['fake_A'].detach().cpu()
    rec_B = visuals['rec_B'].detach().cpu()

    # Create comparison figures
    fig_A2B = create_comparison_figure(real_A[0:1], fake_B[0:1], rec_A[0:1],
                                 title=f"Normal to {config.condition.capitalize()} (Epoch {epoch})")
    fig_B2A = create_comparison_figure(real_B[0:1], fake_A[0:1], rec_B[0:1],
                                 title=f"{config.condition.capitalize()} to Normal (Epoch {epoch})")

    # Save figures
    a2b_path = os.path.join(sample_dir, f"epoch_{epoch}_normal_to_{config.condition}.png")
    b2a_path = os.path.join(sample_dir, f"epoch_{epoch}_{config.condition}_to_normal.png")

    fig_A2B.savefig(a2b_path)
    fig_B2A.savefig(b2a_path)

    plt.close(fig_A2B)
    plt.close(fig_B2A)

    print(f"Saved sample images to {a2b_path} and {b2a_path}")



In [ ]:
# Main function to run training
# Setup configuration
config = Config()
experiment_checkpoint_dir, experiment_sample_dir = config.setup()

# Set up data transforms
train_transform_opt = type('', (), {})()
train_transform_opt.split = 'train'
train_transform_opt.image_size = config.image_size

val_transform_opt = type('', (), {})()
val_transform_opt.split = 'val'
val_transform_opt.image_size = config.image_size

train_transform = get_transforms(train_transform_opt)
val_transform = get_transforms(val_transform_opt)

# Create datasets
train_dataset = ACDCWeatherDataset(
    root_dir=config.dataset_root,
    split='train',
    condition=config.condition,
    transform=train_transform
)

val_dataset = ACDCWeatherDataset(
    root_dir=config.dataset_root,
    split='val',
    condition=config.condition,
    transform=val_transform
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=config.num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=config.num_workers,
    pin_memory=True
)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Create model
model = CycleGANModel(config)

# Train model
train_model(model, train_loader, val_loader, config, experiment_checkpoint_dir, experiment_sample_dir)


Training dataset size: 400
Validation dataset size: 100


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


...

Epoch 200/200:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 200/200 completed in 77.89s | Avg G Loss: 1.0604, Avg D Loss: 0.2685 | LR_G: 0.000002, LR_D: 0.000002


Validation:   0%|          | 0/100 [00:00<?, ?it/s]

Validation - Avg G Loss: 3.4117, Avg D Loss: 0.4176, Avg Cycle Loss: 1.6618
Checkpoint saved to ./checkpoints/CycleGAN-ACDC-Adverse-Weather/epoch_200.pth
Saved sample images to ./samples/CycleGAN-ACDC-Adverse-Weather/epoch_200_normal_to_snow.png and ./samples/CycleGAN-ACDC-Adverse-Weather/epoch_200_snow_to_normal.png


avg_loss_D,█▄▃▃▃▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
avg_loss_G,▃▄▂▂▂▁▁▁▃▃▂▂▂▂▂▄▄▃▂▁▃▃▃▄▂▂▄▂▃▃▄▆▅▅▆▇▇▇██
epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
iter,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇███
loss_D_A,█▄▄▅▃▂▁▆▃▃▆▅▆▅▃▂▅▄▅▅▂▁▂▄▄▅▂▁▇▆▄█▁▂▁▂▂▃▃▄
loss_D_B,▄▄▄▃█▂▄▁▂▂▅▂▂▂▅▂▆▃▁▅▄▁▅▃▃▅▂▅▄▃▃▂▃▄▃▂▂▂▃▃
loss_G_A2B,▄▃▄▃▅▅▅▄▅▄▅▃▃▄▃█▅▂▄▄▆▂▃▅▃█▅▂▆▅▄▃▆▁▆▆▃▄▃▃
loss_G_B2A,█▂▄▃▃▃▃▄▁▂▂▃▄▃▅▅▃▇▅▂▅▁▅▄▂▂▃▅▄▅▅▂▃▂▄▄▄▃▄▃
loss_cycle_A,█▄▄▆▃▂▆▄▄▄▅▄▅▅▅▂▃▆▁▂▅▄▂▁▂▂▂▁▂▁▃▃▁▂▂▂▁▂▁▂
loss_cycle_B,▆▃▂▅▃▄▁▃█▃▄▃▃▅▇▆▅▂▂▄▅▅▄▃▇▂▃▂▁▄▇▂▄▂▂▄▅▂▃▁
loss_identity_A,▆▄█▄▃▃▄▃▅▃▄▃▃▃▃▆▃▃▃▂▃▂▁▆▃▄▃▄▃▃▂▂▂▃▂▂▂▃▁▁


## 5. Inference and Testing
### 5.1 Basic Inference Function
The basic inference function:

- Loads a trained model from a checkpoint
- Processes images from the training and validation sets
- Generates translations in both directions (normal ↔ adverse)
- Saves the results in organized directories for analysis



In [ ]:

def inference(config, checkpoint_path, output_dir):
    """
    Perform inference on training and validation sets and save the translated images.

    Args:
        config: Configuration object
        checkpoint_path: Path to the model checkpoint
        output_dir: Base directory where generated images will be saved
    """
    # Create output directories
    normal_to_adverse_dir = os.path.join(output_dir, f'normal_to_{config.condition}')
    adverse_to_normal_dir = os.path.join(output_dir, f'{config.condition}_to_normal')
    os.makedirs(normal_to_adverse_dir, exist_ok=True)
    os.makedirs(adverse_to_normal_dir, exist_ok=True)

    # Initialize the model
    model = CycleGANModel(config)

    # Load the model checkpoint
    if not model.load_networks(checkpoint_path):
        print(f"Failed to load checkpoint from {checkpoint_path}")
        return

    # Set model to evaluation mode
    model.netG_A2B.eval()
    model.netG_B2A.eval()

    # Create datasets and dataloaders
    transform = transforms.Compose([
        transforms.Resize((config.image_size, config.image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Create datasets
    train_dataset = ACDCWeatherDataset(
        root_dir=config.dataset_root,
        split='train',
        condition=config.condition,
        transform=transform
    )

    val_dataset = ACDCWeatherDataset(
        root_dir=config.dataset_root,
        split='val',
        condition=config.condition,
        transform=transform
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=1,  # Process one image at a time for inference
        shuffle=False,
        num_workers=config.num_workers
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=config.num_workers
    )

    # Process training set
    print(f"Processing {len(train_dataset)} training images...")
    process_dataset(model, train_loader, normal_to_adverse_dir, adverse_to_normal_dir, 'train')

    # Process validation set
    print(f"Processing {len(val_dataset)} validation images...")
    process_dataset(model, val_loader, normal_to_adverse_dir, adverse_to_normal_dir, 'val')

    print(f"Inference completed. Results saved to {output_dir}")

def process_dataset(model, dataloader, normal_to_adverse_dir, adverse_to_normal_dir, split):
    """
    Process a dataset and save generated images.

    Args:
        model: CycleGAN model
        dataloader: DataLoader for the dataset
        normal_to_adverse_dir: Directory to save normal → adverse weather images
        adverse_to_normal_dir: Directory to save adverse weather → normal images
        split: Dataset split ('train' or 'val')
    """
    with torch.no_grad():  # No need to track gradients for inference
        for batch_idx, batch_data in enumerate(tqdm(dataloader, desc=f"Processing {split} set")):
            # Get original image paths
            a_path = batch_data['A_path'][0]  # Normal image path
            b_path = batch_data['B_path'][0]  # Adverse weather image path

            # Generate filename from original path
            a_filename = os.path.basename(a_path)
            b_filename = os.path.basename(b_path)

            # Set model input
            model.set_input(batch_data)

            # Forward pass only
            model.forward()

            # Get generated images
            fake_B = model.fake_B  # Normal → Adverse (A → B)
            fake_A = model.fake_A  # Adverse → Normal (B → A)

            # Save generated images
            # A → B (Normal to Adverse)
            save_path_a2b = os.path.join(normal_to_adverse_dir, f"{split}_{a_filename}")
            save_generated_image(fake_B, save_path_a2b)

            # B → A (Adverse to Normal)
            save_path_b2a = os.path.join(adverse_to_normal_dir, f"{split}_{b_filename}")
            save_generated_image(fake_A, save_path_b2a)

            if batch_idx % 100 == 0:
                print(f"Processed {batch_idx} images")

def save_generated_image(tensor, save_path):
    """
    Convert a tensor to an image and save it.

    Args:
        tensor: PyTorch tensor of shape [1, C, H, W]
        save_path: Path to save the image
    """
    # Ensure the tensor is in CPU
    image = tensor.detach().cpu()

    # Denormalize from [-1, 1] to [0, 1]
    image = (image + 1) / 2.0

    # Convert to PIL Image and save
    image = image[0]  # Remove batch dimension
    image_pil = transforms.ToPILImage()(image)
    image_pil.save(save_path)

# Example usage
def run_inference():
    # Initialize configuration
    config = Config()
    Config.setup()

    # Set the path to your best checkpoint
    checkpoint_path = os.path.join(config.checkpoint_dir, config.experiment_name, 'best_model.pth')

    # Set output directory
    output_dir = f'./inference_results/{config.experiment_name}/{config.condition}'

    # Run inference
    inference(config, checkpoint_path, output_dir)

if __name__ == "__main__":
    run_inference()

### 5.2 High-Resolution Inference
The high-resolution inference implementation:

- Uses a tiling strategy to process large images that wouldn't fit in GPU memory
- Creates overlapping tiles from input images
- Processes each tile independently through the generator
- Blends tiles seamlessly using Gaussian weights for smooth transitions
- Supports both normal → adverse and adverse → normal translations

### 5.3 Visualization and Analysis Tools
Additional tools for model analysis:

- Tile visualization to understand the tiling strategy
- Functions for batch processing multiple images
- Support for preserving directory structures when processing datasets
- Quality assessment of generated images



In [ ]:
# High-Resolution Inference with Tiling Strategy for CycleGAN

import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

class HighResInference:
    def __init__(self, model, tile_size=256, overlap=64, device='cuda'):
        """
        High-resolution inference using tiling strategy for CycleGAN

        Args:
            model: The CycleGANModel instance (already loaded with weights)
            tile_size: Size of each tile (default: 256 to match training)
            overlap: Overlap between tiles for smooth blending (default: 64)
            device: Device to run inference on
        """
        self.model = model
        self.tile_size = tile_size
        self.overlap = overlap
        self.device = device
        self.stride = tile_size - overlap

        # Set model to evaluation mode
        self.model.netG_A2B.eval()
        self.model.netG_B2A.eval()

    def preprocess_image(self, image_path):
        """
        Load and preprocess image for inference

        Args:
            image_path: Path to the input image

        Returns:
            tensor: Preprocessed image tensor
            pil_image: Original PIL image for saving
        """
        # Load image
        pil_image = Image.open(image_path).convert('RGB')

        # Convert to tensor and normalize
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        tensor = transform(pil_image).unsqueeze(0).to(self.device)

        return tensor, pil_image

    def create_tiles(self, image_tensor):
        """
        Create overlapping tiles from the input image

        Args:
            image_tensor: Input image tensor (1, C, H, W)

        Returns:
            tiles: List of tile tensors
            positions: List of tile positions (for reconstruction)
            padding: Padding applied to make dimensions divisible
        """
        _, _, h, w = image_tensor.shape

        # Calculate required padding to make image dimensions divisible by stride
        pad_h = 0 if h % self.stride == 0 else self.stride - (h % self.stride)
        pad_w = 0 if w % self.stride == 0 else self.stride - (w % self.stride)

        # Add padding if necessary
        if pad_h > 0 or pad_w > 0:
            image_tensor = F.pad(image_tensor, (0, pad_w, 0, pad_h), mode='reflect')

        _, _, h_padded, w_padded = image_tensor.shape

        tiles = []
        positions = []

        # Extract tiles
        for y in range(0, h_padded - self.overlap, self.stride):
            for x in range(0, w_padded - self.overlap, self.stride):
                # Ensure tile doesn't exceed image boundaries
                y_end = min(y + self.tile_size, h_padded)
                x_end = min(x + self.tile_size, w_padded)

                # Extract tile
                tile = image_tensor[:, :, y:y_end, x:x_end]

                # Pad if tile is smaller than tile_size (edge case)
                if tile.shape[-1] < self.tile_size or tile.shape[-2] < self.tile_size:
                    pad_h_tile = self.tile_size - tile.shape[-2]
                    pad_w_tile = self.tile_size - tile.shape[-1]
                    tile = F.pad(tile, (0, pad_w_tile, 0, pad_h_tile), mode='reflect')

                tiles.append(tile)
                positions.append((y, x))

        return tiles, positions, (pad_h, pad_w)

    def blend_tiles(self, tiles, positions, original_shape, padding, output_shape):
        """
        Blend overlapping tiles back into a single image

        Args:
            tiles: List of processed tile tensors
            positions: List of tile positions
            original_shape: Original image shape (H, W)
            padding: Padding that was applied
            output_shape: Shape of the padded image

        Returns:
            blended_image: Reconstructed image tensor
        """
        pad_h, pad_w = padding
        h_padded, w_padded = output_shape

        # Create output tensor and weight tensor for blending
        blended = torch.zeros(1, 3, h_padded, w_padded).to(self.device)
        weights = torch.zeros(1, 1, h_padded, w_padded).to(self.device)

        # Create Gaussian weight map for smooth blending
        weight_map = self.create_gaussian_weight_map(self.tile_size).to(self.device)

        # Place tiles and accumulate weights
        for tile, (y, x) in zip(tiles, positions):
            y_end = min(y + self.tile_size, h_padded)
            x_end = min(x + self.tile_size, w_padded)

            # Get the actual size of the tile (might be smaller for edge tiles)
            actual_h = y_end - y
            actual_w = x_end - x

            # Apply weight map (cropped for edge tiles)
            tile_weight = weight_map[:, :, :actual_h, :actual_w]

            # Accumulate tile and weights
            blended[:, :, y:y_end, x:x_end] += tile[:, :, :actual_h, :actual_w] * tile_weight
            weights[:, :, y:y_end, x:x_end] += tile_weight

        # Normalize by weights
        blended = blended / (weights + 1e-8)

        # Remove padding
        if pad_h > 0 or pad_w > 0:
            blended = blended[:, :, :h_padded-pad_h, :w_padded-pad_w]

        return blended

    def create_gaussian_weight_map(self, size):
        """
        Create a Gaussian weight map for smooth tile blending

        Args:
            size: Size of the weight map (tile_size)

        Returns:
            weight_map: Gaussian weight map tensor
        """
        # Create 1D Gaussian window
        window_1d = torch.exp(-0.5 * ((torch.arange(size) - size//2) / (size/6))**2)

        # Create 2D Gaussian by outer product
        window_2d = window_1d.unsqueeze(0) * window_1d.unsqueeze(1)

        # Expand to (1, 1, H, W) format
        weight_map = window_2d.unsqueeze(0).unsqueeze(0)

        return weight_map

    def infer_A2B(self, image_path, output_path=None):
        """
        Convert normal weather image to adverse weather (A to B)

        Args:
            image_path: Path to input image
            output_path: Path to save output image (optional)

        Returns:
            output_image: Generated image as PIL Image
        """
        with torch.no_grad():
            # Load and preprocess image
            image_tensor, original_pil = self.preprocess_image(image_path)

            # Create tiles
            tiles, positions, padding = self.create_tiles(image_tensor)

            # Process each tile
            processed_tiles = []
            for tile in tqdm(tiles, desc="Processing tiles"):
                # Generate adverse weather version
                fake_B = self.model.netG_A2B(tile)
                processed_tiles.append(fake_B)

            # Blend tiles back together
            blended = self.blend_tiles(processed_tiles, positions,
                                     (image_tensor.shape[2], image_tensor.shape[3]),
                                     padding,
                                     (image_tensor.shape[2] + padding[0],
                                      image_tensor.shape[3] + padding[1]))

            # Convert to PIL Image
            output_tensor = blended.squeeze(0).cpu()
            output_np = (output_tensor + 1) / 2.0  # Denormalize
            output_np = output_np.permute(1, 2, 0).numpy()
            output_np = np.clip(output_np, 0, 1)
            output_image = Image.fromarray((output_np * 255).astype(np.uint8))

            # Save if output path provided
            if output_path:
                output_image.save(output_path)
                print(f"Output saved to {output_path}")

            return output_image

    def infer_B2A(self, image_path, output_path=None):
        """
        Convert adverse weather image to normal weather (B to A)

        Args:
            image_path: Path to input image
            output_path: Path to save output image (optional)

        Returns:
            output_image: Generated image as PIL Image
        """
        with torch.no_grad():
            # Load and preprocess image
            image_tensor, original_pil = self.preprocess_image(image_path)

            # Create tiles
            tiles, positions, padding = self.create_tiles(image_tensor)

            # Process each tile
            processed_tiles = []
            for tile in tqdm(tiles, desc="Processing tiles"):
                # Generate normal weather version
                fake_A = self.model.netG_B2A(tile)
                processed_tiles.append(fake_A)

            # Blend tiles back together
            blended = self.blend_tiles(processed_tiles, positions,
                                     (image_tensor.shape[2], image_tensor.shape[3]),
                                     padding,
                                     (image_tensor.shape[2] + padding[0],
                                      image_tensor.shape[3] + padding[1]))

            # Convert to PIL Image
            output_tensor = blended.squeeze(0).cpu()
            output_np = (output_tensor + 1) / 2.0  # Denormalize
            output_np = output_np.permute(1, 2, 0).numpy()
            output_np = np.clip(output_np, 0, 1)
            output_image = Image.fromarray((output_np * 255).astype(np.uint8))

            # Save if output path provided
            if output_path:
                output_image.save(output_path)
                print(f"Output saved to {output_path}")

            return output_image

    def visualize_tiles(self, image_path, output_path=None):
        """
        Visualize the tiling strategy on an image

        Args:
            image_path: Path to input image
            output_path: Path to save visualization (optional)
        """
        # Load image
        image_tensor, pil_image = self.preprocess_image(image_path)

        # Create tiles
        tiles, positions, padding = self.create_tiles(image_tensor)

        # Create visualization
        img_array = np.array(pil_image)
        h, w = img_array.shape[:2]

        # Draw tile boundaries
        for y, x in positions:
            cv2.rectangle(img_array, (x, y),
                         (min(x + self.tile_size, w), min(y + self.tile_size, h)),
                         (255, 0, 0), 2)

        # Create PIL image
        viz_image = Image.fromarray(img_array)

        # Save if output path provided
        if output_path:
            viz_image.save(output_path)
            print(f"Tile visualization saved to {output_path}")

        return viz_image

# Usage example
def load_model_for_inference(checkpoint_path, config):
    """
    Load a trained CycleGAN model for inference

    Args:
        checkpoint_path: Path to the saved model checkpoint
        config: Configuration object

    Returns:
        model: Loaded CycleGAN model
    """
    # Create model
    model = CycleGANModel(config)

    # Load checkpoint
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=config.device)
        model.netG_A2B.load_state_dict(checkpoint['netG_A2B'])
        model.netG_B2A.load_state_dict(checkpoint['netG_B2A'])
        print(f"Model loaded from {checkpoint_path}")
    else:
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

    return model

# Example usage script
if __name__ == "__main__":
    # Configuration
    config = Config()
    config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load trained model
    checkpoint_path = "./checkpoints/CycleGAN-ACDC-Adverse-Weather/best_model.pth"
    model = load_model_for_inference(checkpoint_path, config)

    # Create high-resolution inference handler
    hi_res_inference = HighResInference(
        model=model,
        tile_size=256,    # Match training size
        overlap=64,       # Overlap for smooth blending
        device=config.device
    )

    # # Example 1: Convert normal weather to adverse weather (large image)
    # input_image = "/content/extracted_dataset/rgb_anon/snow/train_ref/GOPR0122/GOPR0122_frame_000161_rgb_ref_anon.png"
    # output_image = "high_res_adverse_output.jpg"
    # result = hi_res_inference.infer_A2B(input_image, output_image)

    # # Example 2: Convert adverse weather back to normal weather
    # input_adverse = "high_res_adverse_image.jpg"
    # output_normal = "high_res_normal_output.jpg"
    # result = hi_res_inference.infer_B2A(input_adverse, output_normal)

    # Example 3: Visualize tiling strategy
    # hi_res_inference.visualize_tiles(input_image, "tiling_visualization.jpg")

    # # Example 4: Process multiple images
    # Process multiple images from subdirectories
    import glob

    input_folder = "/content/extracted_dataset/rgb_anon/snow/val_ref"
    output_folder = "./inference_ref"

    os.makedirs(output_folder, exist_ok=True)

    # Use recursive search with **/* pattern to find all images in subdirectories
    for img_path in glob.glob(f"{input_folder}/**/*.png", recursive=True):
        # Preserve the subdirectory structure
        rel_path = os.path.relpath(img_path, input_folder)
        output_path = os.path.join(output_folder, rel_path)

        # Create subdirectories in output folder if needed
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Convert to adverse weather
        hi_res_inference.infer_A2B(img_path, output_path)
        print(f"Processed: {img_path} -> {output_path}")

Model loaded from ./checkpoints/CycleGAN-ACDC-Adverse-Weather/best_model.pth


Processing tiles: 100%|██████████| 60/60 [00:01<00:00, 48.91it/s]


Output saved to ./inference_ref/GOPR0607/GOPR0607_frame_000474_rgb_ref_anon.png
Processed: /content/extracted_dataset/rgb_anon/snow/val_ref/GOPR0607/GOPR0607_frame_000474_rgb_ref_anon.png -> ./inference_ref/GOPR0607/GOPR0607_frame_000474_rgb_ref_anon.png


Processing tiles: 100%|██████████| 60/60 [00:00<00:00, 129.29it/s]


Output saved to ./inference_ref/GOPR0607/GOPR0607_frame_000410_rgb_ref_anon.png
Processed: /content/extracted_dataset/rgb_anon/snow/val_ref/GOPR0607/GOPR0607_frame_000410_rgb_ref_anon.png -> ./inference_ref/GOPR0607/GOPR0607_frame_000410_rgb_ref_anon.png


...

Output saved to ./inference_ref/GOPR0606/GOPR0606_frame_000258_rgb_ref_anon.png
Processed: /content/extracted_dataset/rgb_anon/snow/val_ref/GOPR0606/GOPR0606_frame_000258_rgb_ref_anon.png -> ./inference_ref/GOPR0606/GOPR0606_frame_000258_rgb_ref_anon.png


Processing tiles: 100%|██████████| 60/60 [00:00<00:00, 125.90it/s]


Output saved to ./inference_ref/GOPR0606/GOPR0606_frame_000213_rgb_ref_anon.png
Processed: /content/extracted_dataset/rgb_anon/snow/val_ref/GOPR0606/GOPR0606_frame_000213_rgb_ref_anon.png -> ./inference_ref/GOPR0606/GOPR0606_frame_000213_rgb_ref_anon.png


Processing tiles: 100%|██████████| 60/60 [00:00<00:00, 126.23it/s]


Output saved to ./inference_ref/GOPR0606/GOPR0606_frame_000220_rgb_ref_anon.png
Processed: /content/extracted_dataset/rgb_anon/snow/val_ref/GOPR0606/GOPR0606_frame_000220_rgb_ref_anon.png -> ./inference_ref/GOPR0606/GOPR0606_frame_000220_rgb_ref_anon.png


## 6. Future Improvements and Applications
### 6.1 Potential Enhancements
Possible improvements to the current implementation:

Incorporation of attention mechanisms for better detail preservation
Experimentation with different loss weightings and architectures
Integration of perceptual losses for enhanced visual quality
Exploration of adaptive normalization techniques

1. Practical Applications
Real-world applications of adverse weather translation:

Data augmentation for autonomous driving systems
Testing robustness of perception algorithms under adverse conditions
Creating synthetic adverse weather datasets for training
Image enhancement and restoration for poor weather conditions

2. Extension to Other Domains
This CycleGAN implementation can be adapted to other image translation tasks:

Style transfer between different artistic domains
Season transfer (summer ↔ winter)
Day-to-night conversion
Cross-domain medical image translation